<a href="https://colab.research.google.com/github/walidbrini/TensorFlow/blob/main/DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

# Load Train and Test data

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

573/573 [==============================] - 0s 0us/step


# View the data

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


#Testing the dataset.batch method


In [ ]:
dataset = tf.data.Dataset.range(8)
for element in dataset:
    print(element.numpy())

dataset = dataset.batch(3)
list(dataset.as_numpy_iterator())

<_RangeDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
0
1
2
3
4
5
6
7


[array([0, 1, 2]), array([3, 4, 5]), array([6, 7])]

# Create an input function

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)


In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


When we use steps : regardless of how many epochs it took to complete
We could have used epochs

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.


In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print("Accuracy equal " + str(eval_result['accuracy']))
#print("Accuracy equal {:.3f}".format(eval_result['accuracy']))



Accuracy equal 0.96666664


In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Run predictions on the test data
predictions = list(classifier.predict(input_fn=lambda: input_fn(test)))

# Iterate through predictions and print class_ids and probabilities
for prediction in predictions:
    class_id = prediction['class_ids'][0]
    probability = prediction['probabilities'][class_id]
    print(f"Predicted class: {class_id}, Probability: {probability:.4f}")


Predicted class: 1, Probability: 0.6290
Predicted class: 2, Probability: 0.6195
Predicted class: 0, Probability: 0.9155
Predicted class: 1, Probability: 0.5172
Predicted class: 1, Probability: 0.6205
Predicted class: 1, Probability: 0.7323
Predicted class: 0, Probability: 0.9879
Predicted class: 2, Probability: 0.5432
Predicted class: 1, Probability: 0.6180
Predicted class: 2, Probability: 0.7676
Predicted class: 2, Probability: 0.7418
Predicted class: 0, Probability: 0.9497
Predicted class: 2, Probability: 0.7852
Predicted class: 1, Probability: 0.6128
Predicted class: 1, Probability: 0.7468
Predicted class: 0, Probability: 0.9641
Predicted class: 1, Probability: 0.7625
Predicted class: 0, Probability: 0.8764
Predicted class: 0, Probability: 0.9499
Predicted class: 2, Probability: 0.7950
Predicted class: 0, Probability: 0.9327
Predicted class: 2, Probability: 0.5081
Predicted class: 2, Probability: 0.7863
Predicted class: 1, Probability: 0.4815
Predicted class: 1, Probability: 0.7440


# Save the model